In [2]:
# 예제 6: 한국관광공사의 대한민국 구석구석사이트에서 특정 지역의 추천 여행지 관련 데이터 크롤링
# https://korean.visitkorea.or.kr/main/main.html

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("예제 6: 대한민국 구석구석 사이트의 여행지 정보 수집하기")
print("=" *80)
query_txt = '대한민국구석구석'
query_area = input('''
 1.서울      2.인천      3.대전      4.대구      5.광주      6.부산      7.울산
 8.세종      9.경기     10.강원     11.충북     12.충남     13.경북     14.경남
15.전북     16.전남     17.제주     18.전체보기 

1.위 지역 중 조회하고 싶은 지역의 번호를 입력해 주세요:   ''')

if query_area == '1' :
    a_value="1"
elif query_area == '2' :
    a_value="2"
elif query_area == '3' :
    a_value="3"
elif query_area == '4' :
    a_value="4"
elif query_area == '5' :
    a_value="5"
elif query_area == '6' :
    a_value="6"
elif query_area == '7' :
    a_value="7"
elif query_area == '8' :
    a_value="8"
elif query_area == '9' :
    a_value="31"
elif query_area == '10' :
    a_value="32"
elif query_area == '11' :
    a_value="33"
elif query_area == '12' :
    a_value="34"
elif query_area == '13' :
    a_value="35"
elif query_area == '14' :
    a_value="36"
elif query_area == '15' :
    a_value="37"
elif query_area == '16' :
    a_value="38"
elif query_area == '17' :
    a_value="39"
elif query_area == '18' :
    a_value="All"

cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10) #페이지수 계산 

# 학습목표 1: 현재 크롤링 시점의 날짜로 폴더 이름을 자동으로 생성하기

f_dir = input("3.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

# 저장될 파일위치와 이름을 지정합니다
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/main/main.html')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택 #차단을 막기위해 

# 학습목표 2: 드롭박스로 보이는 메뉴를 선택하는 방법을 배운다.
#Step 4. 모든 여행지 링크를 선택합니다

driver.find_element_by_xpath('''//*[@id="btnMenu"]''').click()
driver.find_element_by_link_text('''여행지''').click()
time.sleep(3)

# 지역 버튼 누르기
driver.find_element_by_xpath("""//*[@id="areaselect"]""").click() #메뉴 선택 
element1 = Select(driver.find_element_by_id("areaselect")) #대소 조심 Select
element1.select_by_value(a_value) #value는 이것을 선택해라 

time.sleep(2)

# Step 6: 사용자 요청 건수가 실제 검색 건수보다 많을 경우
# 실제 검색 건수로 리셋하기

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

r_cnt= soup.find('div', class_='total_check').find('span').get_text( )
r_cnt2 = r_cnt.replace(",","")
search_cnt = int(r_cnt2)

if cnt > search_cnt :
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

# Step 7. 페이지를 변경하면서 사용자가 요청한 건수만큼 내용을 추출하여 파일에 저장하기

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 내용 컬럼
no = 1

for x in range(1,page_cnt+1) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
    
    for i in range(1,11):  
        f = open(ff_name, 'a',encoding='UTF-8')
        
        if no > cnt :
            break
            
        #각 게시글의 제목 누르기    
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%s]/div[2]/div/a""" % i).click( ) #제목을 누른다 
        time.sleep(2)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    
        #content_top = content_list.find('div','area_txtView top').find('div','inr_wrap').find('div','inr')
        con_1 = content_list.find('p').get_text()
        print(no , ':  ', con_1)
        
        f.write(str(no) + ': '+ str(con_1) + "\n")
        f.close( )
        
        print("\n")
        no2.append(no)
        contents.append(con_1)
    
        driver.back( )  # 뒤로 돌아가기 기능 (뒤로가기)
        time.sleep(2)
                
        no += 1
        
    if x > page_cnt+1 :
            break
        
    x += 1
    
    if (x % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
    else :
            driver.find_element_by_link_text("""%s""" %x).click() # 다음 페이지번호 클릭    
    time.sleep(2)

# Step 8. 출력 결과를 표(데이터 프레임) 형태로 만들어 csv,xls 형식으로 저장하기
korea = pd.DataFrame()
korea['번호']=no2
korea['내용']=contents
        
# csv 형태로 저장하기
korea.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
korea.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )

예제 6: 대한민국 구석구석 사이트의 여행지 정보 수집하기

 1.서울      2.인천      3.대전      4.대구      5.광주      6.부산      7.울산
 8.세종      9.경기     10.강원     11.충북     12.충남     13.경북     14.경남
15.전북     16.전남     17.제주     18.전체보기 

1.위 지역 중 조회하고 싶은 지역의 번호를 입력해 주세요:   1
2.크롤링 할 건수는 몇건입니까?: 20
3.결과 파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\temp\
전체 검색 결과 건수 : 2460 건
실제 최종 출력 건수 20


크롤링 할 총 페이지 번호:  2
1 페이지 내용 수집 시작합니다 =======================
1 :   대한민국 통신역사의 격세지감 제대로 경험하기!KT가 만든 복합 문화공간으로 최신 IT 기기를 체험할 수 있으며, 다양한 문화공연 등을 감상할 수 있다. KT의 5G기술을 체험할 수 있는 ''일상이상''과 대한민국의 130년 통신사를 한눈에 볼 수 있는 ''Gallery 130''으로 구성되어 있다.


2 :   이비스 스타일 앰배서더 서울 강남은 서울 경제의 중심지인 강남구에 위치하며, 합리적인 가격으로 강남 중심지에 머무르길 원하는 비즈니스맨과 레저 여행객에게 강남 지역의 숙박시설을 제공하고 있다. 아늑한 휴식을 위한 공간으로 심플하게 연출한 객실 317개와 고객의 업무가 이루어지도록 도움이 될 비즈니스 미팅룸, 뷔페식 및 간단한 메뉴 주문도 가능한 레스토랑(La table), 고객 건강을 위해 마련된 체력단련시설, 시설이 완비되어 있는 남성 전용 사우나, 지하 주차시설 등을 갖추고 있다.고객에게 꼭 필요한 시설과 서비스를 제공하는 경영 전략으로 편안한 휴식공간과 친절한 서비스, 질 좋은 음식과 같은 필수적인 부분에서 고객에게 큰 만족을 주며 다른 한편으로는 벨맨, 도어맨, 주차 대행 등의 전형적인 특급 호텔의 서비스들을 과감히